# Project 3 Behavioural Cloning
## Background


## Rationale

## Plan

## Conclusion



### Load Data

In [3]:
import cv2
import csv
import numpy as np

In [10]:
lines = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

lines = lines[1:]
        
images = []
measurements = []
for line in lines:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
X_train = np.array(images)
y_train = np.array(measurements)    


In [14]:
## Split into train and validation

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42)


In [23]:
from __future__ import print_function
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 32
num_classes = 10
epochs = 1
learn_rate = 0.001

# input image dimensions
img_rows, img_cols = 160, 320
input_shape = (img_rows, img_cols, 3)

In [16]:
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_train /= 255
X_valid /= 255
print('x_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_valid.shape[0], 'validation samples')

x_train shape: (5625, 160, 320, 3)
y_train shape: (5625,)
5625 train samples
2411 validation samples


In [21]:
# Save Processed Data
import pickle
with open('X_train.p','wb') as myfile:
    pickle.dump(X_train, myfile)   

with open('X_valid.p','wb') as myfile:
    pickle.dump(X_valid, myfile)   
    
with open('y_train.p','wb') as myfile:
    pickle.dump(y_train, myfile)  

with open('y_valid.p','wb') as myfile:
    pickle.dump(y_valid, myfile)   


MemoryError: 

In [24]:
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='linear'))

# model.compile(loss='mse',
#               optimizer=keras.optimizers.Adam(lr=learn_rate), metrics=['mse'])

# model.fit(X_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(X_valid, y_valid),
#           shuffle=True)

# model.save('model.h5')

Train on 5625 samples, validate on 2411 samples
Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[788736,128]
	 [[Node: Variable_37/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable_37"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Variable_37, Const_81)]]

Caused by op 'Variable_37/Assign', defined at:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.4/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.4/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.4/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()

In [28]:
model = Sequential()
model.add(Conv2D(6, (3, 3), activation='relu', input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam',  metrics=['mse'])
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=7)

model.save('model.h5')


Train on 4500 samples, validate on 1125 samples
Epoch 1/7
4500/4500 [==============================] - 10s - loss: 37.9484 - mean_squared_error: 37.9484 - val_loss: 0.0350 - val_mean_squared_error: 0.0350
Epoch 2/7
4500/4500 [==============================] - 10s - loss: 0.0286 - mean_squared_error: 0.0286 - val_loss: 0.0226 - val_mean_squared_error: 0.0226
Epoch 3/7
4500/4500 [==============================] - 10s - loss: 0.0199 - mean_squared_error: 0.0199 - val_loss: 0.0208 - val_mean_squared_error: 0.0208
Epoch 4/7
4500/4500 [==============================] - 10s - loss: 0.0162 - mean_squared_error: 0.0162 - val_loss: 0.0175 - val_mean_squared_error: 0.0175TA: 6s - loss: 0.0167 - mean_squared_e - ETA: 6s - loss: 0.0166 - mean_squared_error: 0. - ETA: 6s - loss: 0.0166 - mean - - ETA: 2s - loss: 0.0167 - mean_squared_error - ETA: 1s - loss: 0.0167 - mean_squar - ETA: 1s - loss: 0.0167 - mean_squar - ETA: 0s - loss: 0.0165 - mean_squared_error:  - ETA: 0s - loss: 0.0165 - mean_squar
